In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import sep
import math
import keras
from keras.utils import np_utils

import numpy as np
from astropy.io import fits
from astropy.visualization import ZScaleInterval
from astropy import wcs
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import random 
import aplpy
from astroquery.sdss import SDSS

from sklearn.metrics import accuracy_score

np.random.seed(123)  # for reproducibility

Using TensorFlow backend.


Downloading data

In [2]:
data_galaxy = np.load('galaxy_ic4559_images.npy')
labels_galaxy = np.load('galaxy_ic4559_labels.npy')
data_sim = np.load('data_sim_images.npy')
labels_sim = np.load('data_sim_labels.npy')

Spliting data to train set and test set

In [3]:
data_sim_train = data_sim[0:8000, :,:,:]
data_sim_test = data_sim[8000:10000,:,:,:]
labels_train = labels_sim[0:8000]
labels_test = labels_sim[8000:10000]

In [4]:
#we need to transform labels to format (N,2) - now is only (N,1)

In [5]:
labels_train_2 = np_utils.to_categorical(labels_train)
labels_test_2 = np_utils.to_categorical(labels_test)

CNN - double input

In [6]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras import backend as K


In [7]:
inputA = keras.layers.Input(shape=(21,21))
inputB = keras.layers.Input(shape=(21,21))

# the first branch operates on the first input
x = Dense(64, activation="relu")(inputA)
x = Dense(32, activation="relu")(x)
x = Dense(8, activation="relu")(x)
x = Dense(4, activation="relu")(x)
#x = Model(inputs=inputA, outputs=x)
 
# the second branch opreates on the second input
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="relu")(y)
y = Dense(4, activation="relu")(y)
#y = Model(inputs=inputB, outputs=y)
 
# combine the output of the two branches
z = keras.layers.concatenate([x, y], axis=-1)
 
# apply a FC layer and then a regression prediction on the
# combined outputs
z = keras.layers.Flatten()(z)
z= Dense(128, activation='relu')(z)
z= Dense(64, activation='relu')(z)
z= Dense(2, activation='softmax')(z)
 
# our model will accept the inputs of the two branches and
# then output a single value
#model = Model(inputs=[x.input, y.input], outputs=z)
model = Model(inputs=[inputA, inputB], outputs=z)

In [8]:
model.compile(optimizer='adam',
          loss='binary_crossentropy',
          metrics=['accuracy'])

In [9]:
data_A = data_sim[:,:,:,0]
data_B = data_sim[:,:,:,1]
labels2 = np_utils.to_categorical(labels_sim)

In [10]:
data_A.shape

(10000, 21, 21)

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21, 21)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 21, 21)       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 21, 64)       1408        input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 21, 64)       1408        input_2[0][0]                    
____________________________________________________________________________________________

In [12]:
model.fit([data_A, data_B],labels2,batch_size=20, epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
10000/10000 [==============================] - 3s 332us/step - loss: 0.0092 - accuracy: 0.9966
Epoch 2/20
10000/10000 [==============================] - 1s 125us/step - loss: 0.0025 - accuracy: 0.9991
Epoch 3/20
10000/10000 [==============================] - 1s 118us/step - loss: 7.7917e-05 - accuracy: 1.0000
Epoch 4/20
10000/10000 [==============================] - 1s 118us/step - loss: 5.2265e-08 - accuracy: 1.0000
Epoch 5/20
10000/10000 [==============================] - 1s 118us/step - loss: 3.0142e-08 - accuracy: 1.0000
Epoch 6/20
10000/10000 [==============================] - 1s 119us/step - loss: 1.8427e-08 - accuracy: 1.0000
Epoch 7/20
10000/10000 [==============================] - 1s 102us/step - loss: 1.1820e-08 - accuracy: 1.0000
Epoch 8/20
10000/10000 [==============================] - 1s 88us/step - loss: 8.1927e-09 - accuracy: 1.0000
Epoch 9/20
10000/10000 [==========

In [13]:
galaxyA = data_galaxy[:,:,:,0]
galaxyB = data_galaxy[:,:,:,1]

In [19]:
labels3 = model.predict([galaxyA,galaxyB])

In [25]:
labels3 = np.round(labels3)

In [20]:
labels3[100:200]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [16]:
labels3_pred = np.argmax(labels3, axis=-1)

In [17]:
labels3_pred

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [25]:
labels3_true = np.zeros(640)

In [26]:
labels3_true[320:640] = 1

In [26]:
labels3_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [18]:
accuracy_score(labels_galaxy, labels3_pred)

0.5